<a href="https://colab.research.google.com/github/ayushjain1144/semantic-segmentation-IGCAR/blob/master/bert_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 3.4MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/igcar_ps/ /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!git clone https://github.com/google-research/bert

Cloning into 'bert'...
remote: Enumerating objects: 329, done.
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (329/329), 238.99 KiB | 3.41 MiB/s, done.
Resolving deltas: 100% (188/188), done.


In [0]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")



from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

auth.authenticate_user()

log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s : \
    %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
    log.info("Using TPU runtime")
    USE_TPU = True
    TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    
    with tf.Session(TPU_ADDRESS) as session:
        log.info('TPU address is ' + TPU_ADDRESS)
        with open('/content/adc.json', 'r') as f:
          auth_info = json.load(f)
        
        tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
        
       
else:
    log.warning('Not connected to TPU runtime')
    USE_TPU = False

W0628 07:56:49.114778 140479545915264 deprecation_wrapper.py:119] From /content/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0628 07:57:13.865776 140479545915264 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

2019-06-28 07:57:15,062 :     Using TPU runtime
I0628 07:57:15.062064 140479545915264 interactiveshell.py:2882] Using TPU runtime
2019-06-28 07:57:15,071 :     TPU address is grpc://10.114.47.2:8470
I0628 07:57:15.071345 140479545915264 interactiveshell.py:2882] TPU address is grpc://10.114.47.2:8470


In [0]:
%cd /content/
!ls

/content
adc.json  bert	gdrive	sample_data


In [0]:
# vocabulary formation

"""

MODEL_PREFIX = "tokenizer"
VOC_SIZE = 32000
NUM_PLACEHOLDERS = 256
SUBSAMPLE_SIZE = 12800000

SPM_COMMAND = ('--input={} --model_prefix={} '
              '--vocab_size={} --input_sentence_size={} '
              '--shuffle_input_sentence=true '
              '--bos_id=-1 --eos_id=-1').format(
              '1.txt', MODEL_PREFIX,
               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)
spm.SentencePieceTrainer.Train(SPM_COMMAND)
"""

'\n\nMODEL_PREFIX = "tokenizer"\nVOC_SIZE = 32000\nNUM_PLACEHOLDERS = 256\nSUBSAMPLE_SIZE = 12800000\n\nSPM_COMMAND = (\'--input={} --model_prefix={} \'\n              \'--vocab_size={} --input_sentence_size={} \'\n              \'--shuffle_input_sentence=true \'\n              \'--bos_id=-1 --eos_id=-1\').format(\n              \'1.txt\', MODEL_PREFIX,\n               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)\nspm.SentencePieceTrainer.Train(SPM_COMMAND)\n'

In [0]:
!ls

1.txt  adc.json  bert  gdrive  sample_data


In [0]:
#!head -n 10 tokenizer.vocab


In [0]:
"""
def read_sentencepiece_vocab(filepath):
  voc = []
  
  with open(filepath, encoding='utf-8') as foo:
    for line in foo:
      voc.append(line.split("\t")[0])
      
    voc = voc[1:]
    return voc
  
snt_vocab = read_sentencepiece_vocab("{}.vocab".format(MODEL_PREFIX))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))
"""

'\ndef read_sentencepiece_vocab(filepath):\n  voc = []\n  \n  with open(filepath, encoding=\'utf-8\') as foo:\n    for line in foo:\n      voc.append(line.split("\t")[0])\n      \n    voc = voc[1:]\n    return voc\n  \nsnt_vocab = read_sentencepiece_vocab("{}.vocab".format(MODEL_PREFIX))\nprint("Learnt vocab size: {}".format(len(snt_vocab)))\nprint("Sample tokens: {}".format(random.sample(snt_vocab, 10)))\n'

In [0]:
"""
def parse_sentencepiece_token(token):
  if token.startswith("▁"):
    return token[1:]
  else:
    return "##" + token
"""    

'\ndef parse_sentencepiece_token(token):\n  if token.startswith("▁"):\n    return token[1:]\n  else:\n    return "##" + token\n'

In [0]:
#bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

In [0]:
#ctrl_symbols = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
#bert_vocab = ctrl_symbols + bert_vocab

In [0]:
#bert_vocab += ["[UNUSED_{}]".format(i) for i in range(VOC_SIZE - len(bert_vocab))]
#print(len(bert_vocab))

In [0]:
#with open('trained_vocab.txt', "w") as foo:
#  foo.write("\n".join([word for word in bert_vocab]))

In [0]:

from bert import modeling, optimization, tokenization


In [0]:
import bert.run_pretraining
from bert.run_pretraining import input_fn_builder, model_fn_builder


In [0]:
MAX_SEQ_LENGTH = 128
MASKED_LM_PROB = 0.15
MAX_PREDICTIONS = 20
DO_LOWER_CASE = True
PROCESSES = 2
PRETRAINING_DIR = "pretraining_data"

In [0]:
!wc -w 1.txt

682357 1.txt


In [0]:
!mkdir ./shards
!split -a 4 -l 256000 -d '1.txt' ./shards/shard_

In [0]:
!ls ./shards/

shard_0000


In [0]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = '/mydrive/bert_uncased/' + BERT_MODEL
print('****** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR)) 
#!ls BERT_PRETRAINED_DIR

****** BERT pretrained directory: /mydrive/bert_uncased/uncased_L-12_H-768_A-12 *****


In [0]:
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt.*'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
!ls $CHKPT_DIR

/mydrive/bert_uncased/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
/mydrive/bert_uncased/uncased_L-12_H-768_A-12/bert_model.ckpt.index
/mydrive/bert_uncased/uncased_L-12_H-768_A-12/bert_model.ckpt.meta


In [0]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
            "python3 bert/create_pretraining_data.py "
            "--input_file=./shards/{} "
            "--output_file={}/{}.tfrecord "
            "--vocab_file={} "
            "--do_lower_case={} "
            "--max_predictions_per_seq={} "
            "--max_seq_length={} "
            "--masked_lm_prob={} "
            "--random_seed=108 "
            "--dupe_factors=5 ")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}',
                            PRETRAINING_DIR, '{}',
                            VOCAB_FILE,
                            DO_LOWER_CASE,
                            MAX_PREDICTIONS, MAX_SEQ_LENGTH,
                            MASKED_LM_PROB)

print(XARGS_CMD)

ls ./shards/ | xargs -n 1 -P 2 -I{} python3 bert/create_pretraining_data.py --input_file=./shards/{} --output_file=pretraining_data/{}.tfrecord --vocab_file=/mydrive/bert_uncased/uncased_L-12_H-768_A-12/vocab.txt --do_lower_case=True --max_predictions_per_seq=20 --max_seq_length=128 --masked_lm_prob=0.15 --random_seed=108 --dupe_factors=5 


In [0]:
tf.gfile.MkDir(PRETRAINING_DIR)


In [0]:
!$XARGS_CMD

W0628 04:50:07.696030 140126984816512 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:469: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0628 04:50:07.696855 140126984816512 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0628 04:50:07.697026 140126984816512 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0628 04:50:07.697160 140126984816512 deprecation_wrapper.py:119] From /content/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0628 04:50:07.817212 140126984816512 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0628 04:50:07.818064 140126984816512 d

In [0]:
BUCKET_NAME = "ayushjain1144-bucket"
MODEL_DIR = "bert_model"

if not BUCKET_NAME:
  log.warning("Warning: no bucket")



In [0]:
MODEL_DIR = "bert_model"
tf.gfile.MkDir(MODEL_DIR)

In [0]:
# hyperparameters for BERT BASE

bert_base_config = {
    "attention_probs_dropout_prob": 0.1,
    "directionality": "bidi",
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "max_position_embeddings": 512,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "pooler_fc_size": 768,
    "pooler_num_attention_heads": 12,
    "pooler_num_fc_layers": 3,
    "pooler_size_per_head": 128,
    "pooler_type": "first_token_transform",
    "vocab_size": 30522
}



In [0]:

!cp $BERT_CONFIG $MODEL_DIR/
!cp $CHKPT_DIR $MODEL_DIR/
!ls $MODEL_DIR/
    
with open("{}/bert_vocab.txt".format(MODEL_DIR), "w") as vocab:
    vocab_bert = open(VOCAB_FILE, 'r').read()
    vocab.write(vocab_bert)
    

bert_config.json		     bert_model.ckpt.index  bert_vocab
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta   bert_vocab.txt


In [0]:
!cp -r $MODEL_DIR $PRETRAINING_DIR /mydrive/

if BUCKET_NAME:
  !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME
else:
  print("Not able to copy")

Copying file://bert_model/bert_config.json [Content-Type=application/json]...
Copying file://bert_model/bert_model.ckpt.index [Content-Type=application/octet-stream]...
Copying file://bert_model/bert_vocab.txt [Content-Type=text/plain]...
Copying file://bert_model/bert_vocab [Content-Type=application/octet-stream]...
Copying file://pretraining_data/shard_0000.tfrecord [Content-Type=application/octet-stream]...
Copying file://bert_model/bert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://bert_model/bert_model.ckpt.meta [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-ob

In [0]:
#TO DO
#code to transfer latest checkpoint from another directory to new directory



In [0]:
TRAIN_BATCH_SIZE = 128
MAX_PREDICTIONS =20
MAX_SEQ_LENGTH = 128
MASKED_LM_PROB = 0.15

EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 1000
SAVE_CHECKPOINTS_STEPS = 100
NUM_TPU_CORES = 8

BERT_DRIVE_DIR = "{}/{}".format('/mydrive', MODEL_DIR)
DATA_DRIVE_DIR = "{}/{}".format('/mydrive', PRETRAINING_DIR)

if BUCKET_NAME:
  BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
  print("bucket name not found")

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

PATH_TO_CHECKPOINT = os.path.join(BERT_GCS_DIR, "bert_model.ckpt")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)


if INIT_CHECKPOINT == None:
    print("no checkpoint found, loading the default")
    INIT_CHECKPOINT = PATH_TO_CHECKPOINT



CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")
VOCAB_FILE = os.path.join(BERT_GCS_DIR, "bert_vocab.txt")

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR, '*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

no checkpoint found, loading the default


2019-06-28 04:58:30,870 :     Using checkpoint: gs://ayushjain1144-bucket/bert_model/bert_model.ckpt
I0628 04:58:30.870596 139821928843136 interactiveshell.py:2882] Using checkpoint: gs://ayushjain1144-bucket/bert_model/bert_model.ckpt
2019-06-28 04:58:30,873 :     Using 1 data shards
I0628 04:58:30.873953 139821928843136 interactiveshell.py:2882] Using 1 data shards


In [0]:
model_fn = model_fn_builder(
    bert_config = bert_config,
    init_checkpoint= INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=TRAIN_STEPS,
    num_warmup_steps=10,
    use_tpu=True,
    use_one_hot_embeddings=True)


tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
            cluster=tpu_cluster_resolver,
            model_dir=BERT_GCS_DIR,
            save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
            tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
            num_shards=NUM_TPU_CORES,
            per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

train_input_fn = input_fn_builder(
    input_files=input_files,
    max_seq_length=MAX_SEQ_LENGTH,
    max_predictions_per_seq=MAX_PREDICTIONS,
    is_training=True)


2019-06-28 04:58:38,160 :     Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f2a829f9510>) includes params argument, but params are not passed to Estimator.
W0628 04:58:38.160904 139821928843136 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f2a829f9510>) includes params argument, but params are not passed to Estimator.
2019-06-28 04:58:38,164 :     Using config: {'_model_dir': 'gs://ayushjain1144-bucket/bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.111.60.10:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimenta

In [0]:

estimator.train(input_fn=train_input_fn, max_steps = TRAIN_STEPS )

2019-06-28 04:58:46,619 :     Querying Tensorflow master (grpc://10.111.60.10:8470) for TPU system metadata.
I0628 04:58:46.619452 139821928843136 tpu_system_metadata.py:78] Querying Tensorflow master (grpc://10.111.60.10:8470) for TPU system metadata.
2019-06-28 04:58:46,643 :     Found TPU system:
I0628 04:58:46.643442 139821928843136 tpu_system_metadata.py:148] Found TPU system:
2019-06-28 04:58:46,645 :     *** Num TPU Cores: 8
I0628 04:58:46.645551 139821928843136 tpu_system_metadata.py:149] *** Num TPU Cores: 8
2019-06-28 04:58:46,651 :     *** Num TPU Workers: 1
I0628 04:58:46.651442 139821928843136 tpu_system_metadata.py:150] *** Num TPU Workers: 1
2019-06-28 04:58:46,655 :     *** Num TPU Cores Per Worker: 8
I0628 04:58:46.655404 139821928843136 tpu_system_metadata.py:152] *** Num TPU Cores Per Worker: 8
2019-06-28 04:58:46,658 :     *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 986085526709126796)
I0628 04:58:46.658172 13982192884